In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1579229624151_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [2]:
# Install packages
sc.install_pypi_package('h2o-pysparkling-2.4')
sc.install_pypi_package('h2o')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Created wheel for h2o-pysparkling-2.4: filename=h2o_pysparkling_2.4-3.28.0.1.post1-py2.py3-none-any.whl size=101307807 sha256=77f35eff44c56ed36f29255e73a2a427c3b94b47342bffa645375017b48c3147
  Stored in directory: /var/lib/livy/.cache/pip/wheels/0d/2f/d2/7e0c36c53b57ca597430858ab3d72e1a612a10780016ff6765
  Created wheel for tabulate: filename=tabulate-0.8.6-cp36-none-any.whl size=23271 sha256=ab6fa31cb127a65694fbee8f15ea71b160ae8157c134d0bf4535e8402efff4a5
  Stored in directory: /var/lib/livy/.cache/pip/wheels/9c/9b/f4/eb243fdb89676ec00588e8c54bb54360724c06e7fafe95278e
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=08509fb959a382ca951acfefbe2138c04553fb7d9725d032f229bd1248760cc9
  Stored in directory: /var/lib/livy/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=0319434b1607f25c9dacb626c459d6384a5b830be1356d8985bb2a88c30

In [3]:
# Create spark session and h2o contezt
from pysparkling import *
import h2o
import pyspark
ss = pyspark.sql.SparkSession.builder.master('yarn')\
        .config("spark.dynamicAllocation.enabled", True)\
        .getOrCreate()
hc = H2OContext.getOrCreate(ss)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Connecting to H2O server at http://ip-172-31-5-90.us-west-2.compute.internal:54321 ... successful.
--------------------------  ------------------------------------------------------------------
H2O cluster uptime:         09 secs
H2O cluster timezone:       Etc/UCT
H2O data parsing timezone:  UTC
H2O cluster version:        3.28.0.1
H2O cluster version age:    1 month
H2O cluster name:           sparkling-water-livy_application_1579229624151_0001
H2O cluster total nodes:    2
H2O cluster free memory:    18.19 Gb
H2O cluster total cores:    8
H2O cluster allowed cores:  8
H2O cluster status:         accepting new members, healthy
H2O connection url:         http://ip-172-31-5-90.us-west-2.compute.internal:54321
H2O connection proxy:
H2O internal security:      False
H2O API Extensions:         XGBoost, Algos, Amazon S3, AutoML, Core V3, TargetEncoder, Core V4
Python version:             3.6.8 final
--------------------------  -------------------------------------------------------------

In [4]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import desc, col, udf, max as max_, lit
from datetime import datetime
from pyspark.sql.types import DateType, IntegerType
from pyspark import SparkContext, SparkConf
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier
import h2o
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators.xgboost import H2OXGBoostEstimator
from h2o.automl import H2OAutoML

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Instantiate spark session
ss = SparkSession.builder.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Make dataframes from parquets
train = ss.read.parquet('s3://intersession-distcomp/train_df_v2')
test = ss.read.parquet('s3://intersession-distcomp/test_df_v2')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Cachce the datasets
train.cache()
test.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[BibNum: int, ItemCount: bigint, ItemCollection: string, PublicationYear: int, ItemType: string, ItemLocation: string, author_book_counts: bigint, publisher_book_counts: bigint, checkouts_l1m: bigint, checkouts_l2m: bigint, checkouts_l3m: bigint, checkouts_l6m: bigint, checkouts_l9m: bigint, checkouts_l12m: bigint, label: int]

In [8]:
# Get dataframe counts
print('Train count: ', train.count())
print('Test count: ', test.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Train count:  785267
Test count:  787918

In [9]:
# Test with numerical data only
df_train_num = train.drop('ItemCollection','ItemType','ItemLocation')
df_test_num = test.drop('ItemCollection','ItemType','ItemLocation')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Assign features
features = df_train_num.columns[:-1]
label = 'label'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Build Vector Assembler
va = VectorAssembler(outputCol="features", inputCols=features)
train_data_num = va.transform(df_train_num).select('features','label').persist()
test_data_num = va.transform(df_test_num).select('features','label').persist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Logistic regression baseline
lr = LogisticRegression()
lrmodel = lr.fit(train_data_num)
pred_lr = lrmodel.transform(test_data_num)
bceval = BinaryClassificationEvaluator()
print(bceval.getMetricName() + ": " + str(bceval.evaluate(pred_lr)))
print(bceval.setMetricName('areaUnderPR').getMetricName() + ': ' \
          + str(bceval.evaluate(pred_lr)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

areaUnderROC: 0.9289622967006272
areaUnderPR: 0.7874413279533533

In [13]:
# Random forest baseline
rf = RandomForestClassifier(seed=42)
rfmodel = rf.fit(train_data_num)
pred_rf = rfmodel.transform(test_data_num)
bceval = BinaryClassificationEvaluator()
print(bceval.setMetricName('areaUnderROC').getMetricName() + ": " \
      + str(bceval.evaluate(pred_rf)))
print(bceval.setMetricName('areaUnderPR').getMetricName() + ': ' \
      + str(bceval.evaluate(pred_rf)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

areaUnderROC: 0.9355303826822277
areaUnderPR: 0.7964937634089002

In [14]:
# GBT baseline
gbtc = GBTClassifier(seed=42)
gbtc_model = gbtc.fit(train_data_num)
gbtcpredicts = gbtc_model.transform(test_data_num)
bceval = BinaryClassificationEvaluator()
print(bceval.setMetricName('areaUnderROC').getMetricName() + ": " \
      + str(bceval.evaluate(gbtcpredicts)))
print(bceval.setMetricName('areaUnderPR').getMetricName() + ': ' \
      + str(bceval.evaluate(gbtcpredicts)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

areaUnderROC: 0.9413065191090155
areaUnderPR: 0.8030877722690843

In [15]:
# Multilayer perceptron baseline
mlc = MultilayerPerceptronClassifier(layers = [11, 5, 4, 2], 
                                     seed=42)
mlc_model = mlc.fit(train_data_num)
mlcpredicts = mlc_model.transform(test_data_num)
bceval = BinaryClassificationEvaluator()
print(bceval.setMetricName('areaUnderROC').getMetricName() + ": " \
      + str(bceval.evaluate(mlcpredicts)))
print(bceval.setMetricName('areaUnderPR').getMetricName() + ': ' \
      + str(bceval.evaluate(mlcpredicts)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

areaUnderROC: 0.5074160147229152
areaUnderPR: 0.12286222982039685

### Next, we will test h2o algorithms.

In [16]:
# Convert spark dataframes to h2o dataframes
h2o_train = hc.as_h2o_frame(df_train_num)
h2o_test = hc.as_h2o_frame(df_test_num)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# Enum the labels
h2o_train['label'] = h2o_train['label'].asfactor()
h2o_test['label'] = h2o_test['label'].asfactor()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# Assign features
predictors = h2o_train.names[:]
predictors.remove('label')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# AutoML baseline
model_automl = H2OAutoML(max_models=10, seed=42, nfolds=3, 
                         max_runtime_secs=360) 

model_automl.train(x=predictors, y='label', 
                   training_frame=h2o_train,
                   validation_frame=h2o_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

AutoML progress: [
03:03:28.52: AutoML: XGBoost is not available in multi-node cluster; skipping it. See http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html#experimental-features for details.

########################################################] 100%

In [20]:
# Display the AutoML leaderboard
model_automl.leaderboard

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

model_id                                                  auc    logloss     aucpr    mean_per_class_error      rmse        mse
---------------------------------------------------  --------  ---------  --------  ----------------------  --------  ---------
StackedEnsemble_BestOfFamily_AutoML_20200117_030328  0.941862   0.227451  0.68458                 0.167574  0.257141  0.0661217
StackedEnsemble_AllModels_AutoML_20200117_030328     0.941862   0.227451  0.68458                 0.167574  0.257141  0.0661217
DRF_1_AutoML_20200117_030328                         0.941789   0.212575  0.702396                0.166244  0.253459  0.0642414
GLM_1_AutoML_20200117_030328                         0.930672   0.232896  0.638664                0.172281  0.260803  0.0680184

[4 rows x 7 columns]

### From the five models we tested, GBT gave us the highest AUC ROC score.

In [22]:
# Look at confusion matrix
from pyspark.mllib.evaluation import MulticlassMetrics

prediction_label = gbtcpredicts.selectExpr("prediction", "cast(label as double) as label").rdd
metrics = MulticlassMetrics(prediction_label)
confusionMetrics = metrics.confusionMatrix()
print("Confusion Metrics = \n%s" % confusionMetrics)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Confusion Metrics = 
DenseMatrix([[637601.,  24046.],
             [ 45501.,  80770.]])

In [23]:
# Cross-validation for GBT
# model = GBTClassifier(labelCol='label',
#                       featuresCol='features',
#                       seed=42)    

# bceval = BinaryClassificationEvaluator()

# paramGrid = ParamGridBuilder().build()
                          
# cv = CrossValidator(estimator=model, 
#                     estimatorParamMaps=paramGrid,
#                     evaluator=bceval,
#                     numFolds=2)

# cvmodel = cv.fit(train_data_num)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# Print cross-validation results
# bceval = BinaryClassificationEvaluator()
# bceval.setMetricName('areaUnderROC').evaluate(cvmodel.bestModel.transform(h2o_test))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# Stop the instance
sc.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…